# Custome Types

__快速回复__: 什么是链表？

如图所示：

<img src="./asset/types/link-list-c.png" width="800" />

在 Rust 中定义结构体（通用语法）

In [8]:
struct Person {
    name: String,
    location: String,
}

fn main() {
    let thea = Person{
        name: "thea".to_string(),
        location: "xian".to_string(),
    };

    println!("{} lives in {}", thea.name, thea.location)
}

// main()

thea lives in xian


()

怎么在Rust中定义 Node

<img src="asset/types/node-rust.png" width="800"/>

## Box in Rust

- 创建一个 Box，将其放在堆上
- 任何东西都可以放进 Box
- Box 拥有 Box 内的所有东西，当 Box 离开作用域时，Box中的值将被销毁

In [ ]:
struct Node {
    value: i32,
}

fn main() {
    let node = Box::new(Node{value: 1});
    println!("{}", node.value);
}

上面代码的解释如图所示：

<img src= "asset/types/box-node.png" width="800"/>

- node 的类型是 `Box<Node>`
- 在堆上声明和分配 Node
- 变量 `node` 是 `Box<Node>` 的所有者
- 当 `node` 不再使用时，`Box` 会自动被销毁
- 当`Box`被销毁，`Node`对象也被销毁

In [ ]:
struct Node {
    value: i32,
    next: Box<Node>,
}

使用 Node: 一个元素的链表

```rust
struct Node {
    value: i32,
    next: Box<Node>,
}

fn main() {
    let node = Box::new(
        Node {
            value: 1,
            next: /* 相当于空指针吗？ */
        }
    )
}
```

### 回想起 Options

In [ ]:
struct Node {
    value: i32,
    next: Option<Box<Node>>,
}

fn main() {
    let node = Box::new(Node{
        value: 1,
        next: None
    });
}

1. 制作一个长的列表

> 下面的代码不会编译

In [13]:
struct Node {
    value: i32,
    next: Option<Box<Node>>,
}

fn main() {
    let mut first = Box::new(Node{value: 1, next: None});
    let second = Box::new(Node{value: 2, next: None});
    first.next = second;
}

Error: mismatched types

错误信息如上面代码中显示的那样，需要的是一个 `Option` 类型，但是给的是一个 `Box`

根据错误信息，对代码进行如下修改

In [14]:
struct Node {
    value: i32,
    next: Option<Box<Node>>,
}

fn main() {
    let mut first = Box::new(Node{value: 1, next: None});
    let second = Box::new(Node{value: 2, next: None});

    first.next = Some(second); // 目前是 Option<Box<Node>>
}

2. 制作一个更长的列表

> 下面的代码不会被编译

In [16]:
struct Node {
    value: i32,
    next: Option<Box<Node>>,
}

fn main() {
    let mut first = Box::new(Node{value: 1, next: None});
    let mut second = Box::new(Node{value: 2, next: None});

    let third = Box::new(Node{value: 3, next: None});
    first.next = Some(second);
    second.next = Some(third);
}

Error: assign to part of moved value: `*second`

__这里发生了什么？__

如图所示

<img src="./asset/types/link-owns.png" width="800"/>

三个变量分别是三个内存的所有者

`first.next = Some(second)` 会将 `second` 的值进行所有权的转让，之后 `second`无法对数据进行操作

如图所示:

<img src="./asset/types/own-move.png" width="800" />

当进行 `second.next = Some(third);`时，因为无法通过`second`访问`Box<Node>`，所以出错。

如图所示：

<img src="./asset/types/own-move-2.png" width="800" />


所有权的链

- 含义：当`first`节点被 Drop
    * 链表的第一个节点被删除
    * 因此 Node 结构体中的 Option 也会被删除
    * 因此 OPtion 中的 Box 会被删除
    * 因此Box中包含的第二个节点也被删除
- 所有的内容都将被清除
- 但是不能使用`second`访问此节点了
- 这些都是Rust中可能会恼人的问题

In [ ]:
struct Node {
    value: i32,
    next: Option<Box<Node>>,
}

fn main() {
    let mut first = Box::new(Node{value: 1, next: None});
    let mut second = Box::new(Node{value: 2, next: None});
    let third = Box::new(Node{value: 3, next: None});

    // 交换刚才的顺序
    second.next = Some(third);
    first.next = Some(second);
}

打印列表

在 C++ 中，打印列表如图所示

<img src="./asset/types/cpp_list.png" width="800" />

如何将代码翻译成 Rust ？

- Rust 中没有指针，那么 `curr` 应该是什么类型呢？
- 希望curr引用第一个节点作为起点，但是不想让 first 失去对节点的所有权（一旦`curr`不再使用，我们不希望列表被释放）
- 循环的条件是什么？（如何知道何时到达结尾？）

In [ ]:
fn main() {
    let mut first = Box::new(Node{value: 1, next: None});
    let mut second = Box::new(Node{value: 2, next: None});
    let third = Box::new(Node{value: 3, next: None});
    second.next = Some(third);
    first.next = Some(second);

    let mut curr = Some(&first);
    // 等价于
    // let mut curr: Option<&Box<Node>> = Some(&first);
}

- 将 `curr` 设置为可变的，因为我们将要重新赋值
- `curr` 是一个 `Option<&Box<Node>>`
- `Option`：可以是`Some` 或者 `None`
    - 使用`None`暗示到达了列表的末尾
- `&Box<Node>`:
    - 若是`Some`: `<&Box<Node>>`
    - 想要通过引用来获取`Box`
    - `Box`包含了堆分配的节点

In [ ]:
fn main() {
    let mut first = Box::new(Node{value: 1, next: None});
    let mut second = Box::new(Node{value: 2, next: None});
    let third = Box::new(Node{value: 3, next: None});
    second.next = Some(third);
    first.next = Some(second);

    let mut curr = Some(&first);

    while curr.is_some() {
        println!("{}", curr.value); 
        // 不会被编译
        // 因为 curr 是一个 Option类型 -- `.vaule` 是无效的

        // update curr
    }
}

- 如果一个变量`curr`是一个`Some`类型，需要提取它的值。
    - 可以使用 `curr.unwrap()`函数
- 否则，将会 panic。

In [ ]:
fn main() {
    let mut first = Box::new(Node{value: 1, next: None});
    let mut second = Box::new(Node{value: 2, next: None});
    let third = Box::new(Node{value: 3, next: None});
    second.next = Some(third);
    first.next = Some(second);

    let mut curr = Some(&first);

    while curr.is_some() {
        println!("{}", curr.unwrap().value);

        // update curr
    }
}

这里可以放心使用`unwrap`, 因为`is_some()`已经做了判断

**回顾一下**：`Option`, `enum`, `unwrap`

`println!("{}", curr.unwrap().value);`

- `curr` 是一个 Option
- Option 是一个 Rust 标准库中的 `enum` 类型
    - 枚举是一种类型，它可以采用特定的、有限数量的定义变量。
    - Rust中，枚举类型可以存储值
- 一个 Option 可以是 `Some` 或者 `None`
- 若是 `Some`, 它存储一个对象（这里是: &Box<Node>）
- `curr.unwrap()` 意味着：
    - 若 `curr` 是 Some，返回 Some 中包含的东西
    - 若 `curr` 是 None，panic

In [ ]:
fn main() {
    let mut first = Box::new(Node{value: 1, next: None});
    let mut second = Box::new(Node{value: 2, next: None});
    let third = Box::new(Node{value: 3, next: None});
    second.next = Some(third);
    first.next = Some(second);

    let mut curr = Some(&first);

    while curr.is_some() {
        println!("{}", curr.unwrap().value);

        curr = curr.unwrap().next; // 代码不会被编译成功
    }
}

原因如下：
- `curr.unwrap()`返回了一个 Node 类型
- Node.next 给我们了 Option<Box<Node>> 类型

**介绍 `as_ref()`**

- 转换 `&Option<T>` 为 `Option<&T>`
- 如果给定的 Option 是 None，返回 None

比如：

In [ ]:
let mut curr = Some(&first);
while curr.is_some() {
    println!("{}", curr.unwrap().value);

    curr = (&curr.unwrap().next).as_ref();
}

- `curr.unwrap().next` 返回一个 `Option<Box<Node>>` 类型
- 应用了 `&`, 给我们了 `&Option<Box<Node>>`类型
- 应用 `as_ref` 给我们了 `Option<&Box<Node>>` 类型
- 如果 `curr.unwrap().next` 是 None，`as_ref()` 返回 None